In [324]:
import execute_tensor as execute
from execute_tensor import execute as exe

In [84]:
tox_names = ['hERG','AMES',"Carcinogens_Lagunin",'ClinTox','hERG_Karim']
tokens = ['SMILE','AIS']
BERT_names = []

for token in tokens:
    BERT_names.append(token+"_BERT_Classifier")

for tox_name in tox_names:
    tox_execute = exe(tox_name,0.2,552,epoch=20,batch=32,tokens=tokens)
    tox_execute.forward()
    globals()['tox_execute_'+tox_name] = tox_execute
    
    models = {tox_execute.Bit:'Bit_Classifier',tox_execute.BERTs[0]:BERT_names[0],tox_execute.BERTs[1]:BERT_names[1]}
    execute.plot_history(models,tox_name,['SMILE_AIS'])

Unexpected :  [Cu+2]
Unexpected :  [Mn+2]
Unexpected :  [H-]
Unexpected :  [SbH]
Epoch 1/20


KeyboardInterrupt: 

In [36]:
tox_names = ['hERG','AMES',"Carcinogens_Lagunin",'ClinTox','hERG_Karim']
BERT_names = []

for token in tokens:
    BERT_names.append(token+"_BERT_Classifier")

for tox_name in tox_names:
    tox_execute = exe(tox_name,0.2,552,epoch=20,batch=32,tokens=tokens)
    tox_execute.forward()
    globals()['tox_execute_'+tox_name] = tox_execute
    
    models = {tox_execute.Bit:'Bit_Classifier',tox_execute.BERTs[0]:BERT_names[0],tox_execute.BERTs[1]:BERT_names[1]}
    execute.plot_history(models,tox_name,['SMILE_AIS'])

Epoch 1/20
 93/328 [=======>......................] - ETA: 44s - loss: 0.5295 - acc: 0.7476 - auc: 0.7370

KeyboardInterrupt: 

In [325]:
from Module import custom_layers
import tensorflow as tf

BERT_tensor = custom_layers.BERT_tensor

bert_layer = BERT_tensor(256,6,1024,strat_index=10)

In [326]:
from tensorflow.keras import Model
inputs = tf.keras.layers.Input(shape = (200,),dtype=tf.int32)
outputs = bert_layer(inputs,None)

model = Model(inputs = [inputs], outputs = [outputs])

In [327]:
model.load_weights('./BERT/atomInSmile/small_tensor_Pre_BERT')
paras = model.get_weights()
bert_layer.set_weights(paras)

In [328]:
import numpy as np
def Task_mask(num_task):
    result = np.zeros([200,200])
    for i in range(num_task):
        for j in range(200):
            if j == i:
                continue
            else:
                result[j][i] = 1
    return result

In [329]:
mask = Task_mask(10)

In [330]:
from tensorflow.keras import Model
from tensorflow.keras.metrics import AUC
Models = []

Input = tf.keras.layers.Input(200,)
hidden = bert_layer(Input,att_mask = mask)
output = hidden[:,0]
output = tf.keras.layers.Dense(256,activation = 'gelu')(output)
output = tf.keras.layers.Dense(1)(output)
globals()[f'Task{0}_model'] = Model(inputs = [Input],outputs = [output])
globals()[f'Task{0}_model'].compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5))
Models.append(globals()[f'Task{0}_model'])

for i in range(9):
    i = i+1
    Input = tf.keras.layers.Input(200,)
    hidden = bert_layer(Input,att_mask = mask)
    output = hidden[:,i]
    output = tf.keras.layers.Dense(256,activation = 'gelu')(output)
    output = tf.keras.layers.Dense(1,activation = 'sigmoid')(output)
    globals()[f'Task{i}_model'] = Model(inputs = [Input],outputs = [output])
    globals()[f'Task{i}_model'].compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),loss = 'binary_crossentropy',metrics=['acc',AUC(name='auc')])
    Models.append(globals()[f'Task{i}_model'])
    

In [259]:
import os
import tensorflow as tf 
from tensorflow.python.client import device_lib
import pickle
import numpy as np
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split
from tdc.single_pred import Tox
from Module import RDK


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim,mask_zero=True)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim,mask_zero = True)

    def call(self, x):
        positions = np.array([0]+[0]+[i+2 for i in range(198)])
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions    
    
    
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        l2_reg = tf.keras.regularizers.l2(0.01)
        
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim,kernel_regularizer=l2_reg)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs,attention_mask=tf.cast(np.array([[1]+[0]+[1]*198] + [[0]+[1]*199] + [[0]*2+[1]*198]*198),bool))
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    
    
    
class BERT(tf.keras.layers.Layer):
    def __init__(self,emb_dim,num_heads,ff_dim):
        super(BERT, self).__init__()
        self.encoder = tf.keras.Sequential([TransformerBlock(emb_dim,num_heads,ff_dim) for i in range(8)])
        
        self.embedding = TokenAndPositionEmbedding(200,3500,256)
        self.dense = tf.keras.layers.Dense(250,activation = 'gelu')
        self.classify = tf.keras.layers.Dense(71,activation = 'softmax')
    def call(self, inputs, mask_index=None,pretrain = False):
        if pretrain:
            mask_index = tf.one_hot(mask_index,200)
            boolean_mask = tf.cast(tf.reduce_sum(mask_index,axis=1),bool)
            inputs = tf.cast(inputs,dtype=tf.int32)
        
        inputs = tf.reshape(inputs,[-1,200])
        hidden = self.embedding(inputs)
        
        
        hidden = self.encoder(hidden)
    
        if pretrain:
            output = tf.reshape(hidden,[-1,200,256])
            output = self.dense(output)
            output = layers.Dropout(0.1)(output)
            output = self.classify(output)
            output = tf.boolean_mask(output,boolean_mask)
            return output
        else:
            return hidden


def predict(model,results,len_list):
    index = 0
    res = model.predict(results,verbose=0)
    x_val = []
    for i in len_list:
        temp = res[index:index+i]
        x_val.append(np.average(temp,axis=0))
        index = index+i
    return np.array(x_val)

def similar(a, b):    return SequenceMatcher(None, a, b).ratio()
def most_similar(query,word2idx):

    max = 0
    tokken = ''
    query = query.split(';')
    for i in word2idx.keys():
        key = i.split(';')
        temp2 = 0
        temp3 = 0
        temp1 = similar(query[0],key[0])*10
        try:
            temp2 = similar(query[1],key[1])*2
            temp3 = similar(query[2],key[2])*1
        except:
            pass
        temp = temp1+temp2+temp3
        if temp>max:
            max = temp
            tokken = i
    return tokken

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, x_val,y_true,len_20):
        super().__init__()
        self.x_val = x_val
        index = 0
        res = []
        for i in len_20:
            res.append(np.average(y_true[index:index+i]))
            index = index+i
        self.len20 = len_20
        self.counts = []
        self.max = 0
        self.y_true = np.array(res)
        self.history = {}
        self.epoch = 0
    def on_epoch_end(self, epoch, logs=None):
        # 에포크가 끝날 때마다 validation 데이터로 모델 평가
        result = predict(self.model,self.x_val,self.len20)
        acc = Accuracy()(self.y_true,np.round(result))
        auc_res = (AUC()(self.y_true,result)).numpy()
        loss = tf.keras.metrics.BinaryCrossentropy()(self.y_true,result)
        auc_res = auc_res
        print(f"     val_acc : {acc},    val_auc : {auc_res} val_loss : {loss}")
        if 'val_acc' not in self.history:
            self.history['val_acc'] = [acc]
        else:
            self.history['val_acc'] += [acc]
        if 'val_auc' not in self.history:
            self.history['val_auc'] = [auc_res]
        else:
            self.history['val_auc'] += [auc_res]
        
        if 'val_loss' not in self.history:
            self.history['val_loss'] = [loss]
        else:
            self.history['val_loss'] += [loss]
        
        self.max = np.max(self.history['val_auc'])
            
        if self.history['val_auc'][-1]<self.max:
            self.counts.append(1)
        else:
            self.counts = []

        self.epoch += 1
        """
        if self.epoch>10 and len(self.counts)>2:
            self.model.stop_training = True"""



inputs = Input(shape = (200,),dtype=tf.int32)
outputs = BERT(256,6,1024)(inputs,None)

model = Model(inputs = [inputs], outputs = [outputs])

#model.load_weights('./BERT/atomInSmile/F_Random_ZINC_L_model_weights.h5')


#BERT_parameters = model.get_weights()[:130]



from tensorflow.keras.layers import Input
from tensorflow.keras import Model

def BERT_model():
    inputs = Input(200,)
    hidden = bert_layer(inputs)
    hidden = hidden[:,0]
    hidden = tf.keras.layers.Dense(256,activation = 'gelu')(hidden)
    output = tf.keras.layers.Dense(1,activation = 'sigmoid')(hidden)
    result = Model(inputs = [inputs],outputs = [output])
    return result

def Bit_model():
    inputs = Input(2048,)
    hidden = tf.keras.layers.Dense(250,activation = 'relu')(inputs)
    hidden = tf.keras.layers.Dropout(0.3)(hidden)
    hidden = tf.keras.layers.Dense(40,activation = 'relu')(hidden)
    hidden = tf.keras.layers.Dropout(0.3)(hidden)
    hidden = tf.keras.layers.Dense(10,activation = 'relu')(hidden)
    hidden = tf.keras.layers.Dropout(0.3)(hidden)
    output = tf.keras.layers.Dense(1,activation = 'sigmoid')(hidden)
    result = Model(inputs = [inputs],outputs = [output])
    return result


    
class tox_process():
    def __init__(self,tox,test_size=0.2,random_state = 1024):
        self.tox_name = tox
        self.size = test_size
        self.seed = random_state
        
        
    def AIS_process(self,plot=False,token = 'AIS',number_of_task = 2):
        if token == 'AIS':
            with open('./Tox_data/AIS_Tox_data/'+self.tox_name,'rb') as file:
                train,label,len_20 = pickle.load(file)[0]
            with open('./BERT/atomInSmile/1M_random_ZINC_word2index.pkl','rb') as file:
                word2idx = pickle.load(file)
        elif token == 'SMILE':
            with open('./Tox_data/SMILE_Tox_data/'+self.tox_name,'rb') as file:
                train,label,len_20 = pickle.load(file)[0]
            with open('./BERT/SMILE/1M_random_ZINC_word2index.pkl','rb') as file:
                word2idx = pickle.load(file)
                
            """elif token == 'SmiletoPE':
                with open('./Tox_data/SmiletoPE/'+self.tox_name,'rb') as file:
                    train,label,len_20 = pickle.load(file)[0]
                with open('./BERT/SmiletoPE/1M_random_ZINC_word2index.pkl','rb') as file:
                    word2idx = pickle.load(file)"""
        else:
            raise
        
        
        
        
        
        if plot:
            temp_dict = {}

            for i in train:
                try:
                    temp_dict[len(i)] = temp_dict[len(i)] + 1
                except:
                    temp_dict[len(i)] = 1
                    
            plt.bar(temp_dict.keys(),temp_dict.values())
        
        except_dict = {}

        for i in train:
            for j in i:
                try:
                    word2idx[j]
                except:
                    try:
                        except_dict[j]
                    except:
                        except_dict[j] = len(except_dict) + 1
        
        similar_dict = {}

        for i in except_dict.keys():
            similar_dict[i] = most_similar(i,word2idx)
            
            
        AIS_train = []
        for index,i in enumerate(train):
            
            temp = []
            if number_of_task == 2:
                temp.append(3)
                temp.append(4)
            else:
                for k in range(number_of_task):
                    temp.append(len(word2idx)+1+k)
            temp.append(1)
            for j in i:
                try:
                    temp.append(word2idx[j])
                except:
                    print('Unexpected : ',j)
                    word2idx[j] = len(word2idx)+1
                    temp.append(word2idx[j])
                """    
                except:
                    if j != '/[H]':
                        print(j,i)
                        word_sim = similar_dict[j]
                        if word_sim != '':
                            temp.append(word2idx[word_sim])
                    else:
                        pass"""
            if len(temp)>1:
                AIS_train.append(temp)

        AIS_train = tf.keras.preprocessing.sequence.pad_sequences(AIS_train, padding='post', maxlen=200)
        temp_x = []
        temp_y = []
        index = 0
        for i in len_20:
            temp_x.append(AIS_train[index:index+i])
            temp_y.append(label[index:index+i])
            index = index+i



        x_train, x_val, y_train, y_val,_,len_20 = train_test_split(temp_x,temp_y,len_20, test_size=self.size,random_state=self.seed)
        
        def flatten(data):
            temp = []
            for i in data:
                temp+=list(i)
            data = np.array(temp)
            return data
        x_train = flatten(x_train)
        x_val = flatten(x_val)
        y_val = flatten(y_val)
        y_train = flatten(y_train)
        return x_train, x_val, y_train, y_val,len_20
    
    def bit_precess(self):
        train,tox_info = Tox(name=self.tox_name).get_data(format='DeepPurpose')
        bit_string = RDK.smile_to_RDkit(train,2048)
        x_train_NN,x_val_NN,y_train_NN,y_val_NN = train_test_split(np.array(bit_string)/1.,np.array(tox_info)/1.,test_size=self.size,random_state=self.seed)
        return x_train_NN,x_val_NN,y_train_NN,y_val_NN
    


class execute():
    def __init__(self,tox,test_size,split_seed,epoch = 20,batch=32*20,tokens = ['AIS']):
        super().__init__()
        self.tox = tox
        self.size = test_size
        self.seed = split_seed
        self.epoch = epoch
        self.BERTs = []
        self.batch_size = batch
        Bit_Classifier = Bit_model()
        self.Bit = Bit_Classifier
        #model.load_weights('./BERT/atomInSmile/F_Random_ZINC_L_model_weights.h5')
        self.tokens = tokens
        self.BERT_parameters = []
        
        for token in tokens:
            if token == 'AIS':
                model.load_weights('./BERT/atomInSmile/Pre_BERT.h5')
            elif token == 'SMILE':
                model.load_weights('./BERT/SMILE/Pre_BERT.h5')
            elif token == 'SmiletoPE':
                model.load_weights('./BERT/SmiletoPE/F_Random_ZINC_L_model_weights.h5')
            else:
                raise
            self.BERTs.append(BERT_model())
            self.BERT_parameters.append(model.get_weights()[:130])
    def forward(self,set_weights=True):
        if set_weights:
            for index,token in enumerate(self.tokens):
                BERT_parameter = self.BERT_parameters[index]
                self.BERTs[index].layers[1].set_weights(BERT_parameter)
        

        for index,token in enumerate(self.tokens): 
            process = tox_process(self.tox, self.size, self.seed)
            x_train, x_val, y_train, y_val,len_20 = process.AIS_process(token = token)
            
            temp_BERT = self.BERTs[index]
            
            val_call = CustomCallback(x_val,y_val,len_20)
            
            temp_BERT.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),loss = 'binary_crossentropy',metrics=['acc',AUC(name='auc')])
            
            hist1 = temp_BERT.fit(x_train,y_train,batch_size=self.batch_size,epochs=self.epoch,callbacks=[val_call])
            
        
            temp_BERT.history.history['val_loss'] = val_call.history['val_loss']
            temp_BERT.history.history['val_acc'] = val_call.history['val_acc']
            temp_BERT.history.history['val_auc'] = val_call.history['val_auc']
            
        x_train_NN,x_val_NN,y_train_NN,y_val_NN = process.bit_precess()
        self.Bit.compile(optimizer = 'Adam',loss = 'binary_crossentropy',metrics=['acc',AUC(name='auc')])
        hist2 = self.Bit.fit(x_train_NN,y_train_NN,batch_size=32,epochs=self.epoch,validation_data=(x_val_NN,y_val_NN))
        
        
        
import matplotlib.pyplot as plt
def plot_history(models,tox_name,token=['AIS']):
    plt.figure(figsize=(16,8))
    plt.subplot(2, 3, 1)
    for model in models.keys():
        plt.plot([i for i in range(len(model.history.history['loss']))],model.history.history['loss'],label=models[model])
    #plt.plot([i for i in range(len(val_call2.history.history['val_loss']))],val_call2.history.history['val_loss'],label = 'BERT_Norm acc data')
    plt.title(tox_name + ' train Loss')
    plt.xlabel('epoch')
    plt.ylabel('score')

    plt.legend()
    plt.subplot(2, 3, 2)
    for model in models.keys():
        plt.plot([i for i in range(len(model.history.history['acc']))],model.history.history['acc'],label=models[model])
    #plt.plot([i for i in range(len(val_call2.history.history['val_acc']))],val_call2.history.history['val_acc'],label = 'BERT_Norm acc data')
    plt.title(tox_name + ' train ACC')
    plt.xlabel('epoch')
    plt.ylabel('score')

    plt.legend()
    plt.subplot(2, 3, 3)
    for model in models.keys():
        plt.plot([i for i in range(len(model.history.history['auc']))],model.history.history['auc'],label=models[model])
    plt.title(tox_name + ' train AUC')
    plt.xlabel('epoch')
    plt.ylabel('score')

    plt.subplot(2, 3, 4)
    for model in models.keys():
        plt.plot([i for i in range(len(model.history.history['val_loss']))],model.history.history['val_loss'],label=models[model])
    #plt.plot([i for i in range(len(val_call2.history.history['val_loss']))],val_call2.history.history['val_loss'],label = 'BERT_Norm acc data')
    plt.title(tox_name + ' val Loss')
    plt.xlabel('epoch')
    plt.ylabel('score')

    plt.legend()
    plt.subplot(2, 3, 5)
    for model in models.keys():
        plt.plot([i for i in range(len(model.history.history['val_acc']))],model.history.history['val_acc'],label=models[model])
    #plt.plot([i for i in range(len(val_call2.history.history['val_acc']))],val_call2.history.history['val_acc'],label = 'BERT_Norm acc data')
    plt.title(tox_name + ' val ACC')
    plt.xlabel('epoch')
    plt.ylabel('score')

    plt.legend('epoch')
    plt.ylabel('score')

    plt.legend()
    plt.tight_layout() 
    plt.subplot(2, 3, 6)
    for model in models.keys():
        plt.plot([i for i in range(len(model.history.history['val_auc']))],model.history.history['val_auc'],label=models[model])
    plt.title(tox_name + ' val AUC')
    plt.xlabel('epoch')
    plt.ylabel('score')
    
    plt.savefig(f'./Results/{token[0]}_{token[1]}_Tox_result/'+tox_name+'.png')
    
    plt.show()
    
    
    
    

In [331]:
import random
Tox_names = ['LD50_Zhu','AMES','ClinTox','hERG','DILI','Skin_Reaction','hERG_Karim']
x_trains = [] 
y_trains = []
x_vals = []
y_vals = []
len_20s = []
task_labels = []
val_task = []
for index1,i in enumerate(Tox_names):
    try:
        
        A = tox_process(i)
        x_train,x_val,y_train,y_val,len_20 = A.AIS_process(number_of_task=10,token='AIS')
        index = 0
        res = []
        for i in len_20:
            res.append(np.average(y_val[index:index+i]))
            index = index+i
        y_val = np.array(res)
        
        temp_zip = list(zip(x_train,y_train))
        random.shuffle(temp_zip)
        x_train,y_train = zip(*temp_zip)
        
        
        temp_x_trains = x_train[:int(len(x_train)/32)*32]
        temp_x_trains = np.reshape(temp_x_trains,[-1,32,200])
        x_trains += list(temp_x_trains) + list(x_train[int(len(x_train)/32)*32:])
        
        temp_y_trains = y_train[:int(len(y_train)/32)*32]
        temp_y_trains = np.reshape(temp_y_trains,[-1,32])
        tmp = list(temp_y_trains) + list(y_train[int(len(y_train)/32)*32:])
        y_trains += tmp
        
        
        
        task_labels+=[index1]*len(tmp)
        
        
        y_vals.append(y_val)
        x_vals.append(x_val)
        len_20s.append(len_20)
        val_task.append(index1)
    except:
        pass

In [332]:
import random

temp_zip = list(zip(x_trains,y_trains,task_labels))
random.shuffle(temp_zip)

x_trains,y_trains,task_labels = zip(*temp_zip)

In [333]:
len(x_trains)

15350

In [334]:
len(Tox_names)

7

In [335]:
x_trains[0][0]

array([1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048,    1,
         20,   11,    9,   50,   19,   37,   20,    9,   97,   17,  193,
         19,   37,   37,   37,   11,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [336]:
from tqdm import tqdm
import sys
batch_size = 32
epochs = 25


loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
loss_fn_reg = tf.keras.losses.MeanSquaredError()
acc = np.array([0])



for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loss_list = [[] for i in range(len(Tox_names))]
    acc_list = [[] for i in range(len(Tox_names))]
    auc_list = [[] for i in range(len(Tox_names))]
    for i in range(0, len(temp_zip)):
        batch_images = x_trains[i]
        batch_images = tf.reshape(batch_images,[-1,200])
        batch_labels = y_trains[i]
        batch_labels = tf.reshape(batch_labels,[-1])
        task = task_labels[i]

        if task != 0:
            with tf.GradientTape() as tape:
                model = globals()[f'Task{task}_model']
                logits = model(batch_images)
                logits = tf.reshape(logits,[-1])
                loss_value = loss_fn(batch_labels, logits)
                acc = tf.keras.metrics.Accuracy()(np.round(logits),batch_labels)
                auc = tf.keras.metrics.AUC()(batch_labels,logits)
                auc = auc.numpy()
                loss_value = tf.reduce_mean(loss_value)
                loss_list[task].append(loss_value)
                acc_list[task].append(acc)
                auc_list[task].append(auc)
                
            grads = tape.gradient(loss_value, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        else:
            with tf.GradientTape() as tape:
                model = globals()[f'Task{task}_model']
                logits = model(batch_images)
                logits = tf.reshape(logits,[-1])
                loss_value = loss_fn_reg(logits,batch_labels)
                loss_value = tf.reduce_mean(loss_value)
                loss_list[task].append(loss_value)
                acc = tf.keras.metrics.RootMeanSquaredError(name="root_mean_squared_error", dtype=None)(logits,batch_labels)
                acc_list[task].append(acc)
            
            grads = tape.gradient(loss_value, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
        for l in range(len(Tox_names)):
            
            temp_loss = np.average(loss_list[l][-10:])
            temp_acc = np.average(acc_list[l][-10:])
            temp_auc = np.average(auc_list[l][-10:])
            if l == 0:
                text = "\rSteps : {} Task : {}, Loss: {:.4f}, acc : {:.4f}, auc : {:.4f}".format(i,l,temp_loss,temp_acc,temp_auc)
            else:
                text += "     Task : {}, Loss: {:.4f}, acc : {:.4f}, auc : {:.4f}".format(l,temp_loss,temp_acc,temp_auc)
            sys.stdout.write(text)
            sys.stdout.flush()
    # 각 에포크 종료 후 평가
    for temp_index,j in enumerate(val_task):
        j = j
        val_res = predict(globals()[f'Task{j}_model'],x_vals[temp_index],len_20s[temp_index])
        if j != 0:
            acc = tf.keras.metrics.Accuracy()(y_vals[temp_index],np.round(val_res))
            auc_res = (AUC()(y_vals[temp_index],val_res)).numpy()
            loss = tf.keras.metrics.BinaryCrossentropy()(y_vals[temp_index],val_res)
            
            print(f'\nTask is {Tox_names[temp_index]}')
            print(f"Test accuracy: {acc}")
            print(f"Test AUC: {auc_res}")
            print(f"Test loss: {loss}\n")
        else:
            loss = tf.keras.metrics.MeanSquaredError()(y_vals[temp_index],val_res)
            print(f'\nTask is {Tox_names[temp_index]}')
            print(f"\nTest MSE:{loss}")

Epoch 1/25
Steps : 1460 Task : 0, Loss: 0.7210, acc : 0.8317, auc : nan     Task : 1, Loss: 0.6339, acc : 0.7188, auc : 0.8107     Task : 2, Loss: 0.6988, acc : 0.9281, auc : 0.5363     Task : 3, Loss: 0.5570, acc : 0.7937, auc : 0.8234     Task : 4, Loss: 0.7507, acc : 0.4062, auc : 0.3354     Task : 5, Loss: 0.7151, acc : 0.5688, auc : 0.4020     Task : 6, Loss: 0.6294, acc : 0.6406, auc : 0.7644

In [321]:
loss_fn(batch_labels, logits)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5132855>

In [320]:
logits

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([9.9980849e-01, 9.9999833e-01, 9.9994910e-01, 2.6586196e-07,
       4.6071282e-04, 9.9972433e-01, 2.5018396e-08, 9.9790084e-01,
       9.9999797e-01, 9.9999857e-01, 1.6916485e-07, 9.9999988e-01,
       9.9999177e-01, 7.8884675e-04, 9.9999893e-01, 6.5154715e-08,
       3.8722131e-08, 8.3348788e-02, 1.0000000e+00, 3.4719207e-07,
       1.3504310e-09, 1.0000000e+00, 1.0497830e-02, 9.9999988e-01,
       1.6620055e-05, 9.9997461e-01, 9.9999726e-01, 1.3299459e-07,
       1.0000000e+00, 3.9634336e-04, 8.8064539e-01, 1.0000000e+00],
      dtype=float32)>

In [318]:
logits

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([9.9980849e-01, 9.9999833e-01, 9.9994910e-01, 2.6586196e-07,
       4.6071282e-04, 9.9972433e-01, 2.5018396e-08, 9.9790084e-01,
       9.9999797e-01, 9.9999857e-01, 1.6916485e-07, 9.9999988e-01,
       9.9999177e-01, 7.8884675e-04, 9.9999893e-01, 6.5154715e-08,
       3.8722131e-08, 8.3348788e-02, 1.0000000e+00, 3.4719207e-07,
       1.3504310e-09, 1.0000000e+00, 1.0497830e-02, 9.9999988e-01,
       1.6620055e-05, 9.9997461e-01, 9.9999726e-01, 1.3299459e-07,
       1.0000000e+00, 3.9634336e-04, 8.8064539e-01, 1.0000000e+00],
      dtype=float32)>

In [302]:
for temp_index,j in enumerate(val_task):
    j = j
    val_res = predict(globals()[f'Task{j}_model'],x_vals[temp_index],len_20s[temp_index])
    if j != 0:
        acc = tf.keras.metrics.Accuracy()(y_vals[temp_index],np.round(val_res))
        auc_res = (AUC()(y_vals[temp_index],val_res)).numpy()
        loss = tf.keras.metrics.BinaryCrossentropy()(y_vals[temp_index],val_res)
        
        print(f'\nTask is {Tox_names[temp_index]}')
        print(f"Test accuracy: {acc}")
        print(f"Test AUC: {auc_res}")
        print(f"Test loss: {loss}\n")
    else:
        loss = tf.keras.metrics.MeanSquaredError()(y_vals[temp_index],val_res)
        print(f'\nTask is {Tox_names[temp_index]}')
        print(f"\nTest MSE:{loss}")


Task is 
Test accuracy: 0.821052610874176
Test AUC: 0.8632075786590576
Test loss: 0.666398286819458

